In [1]:
import re
import json
import time
import pickle
from collections import defaultdict

import spacy
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
%%time
OPTIONS = Options()
OPTIONS.add_argument('--headless')
# make sure Homebrew is installed
# then run 'brew tap homebrew/cask && brew cask install chromedriver'
driver = webdriver.Chrome(options=OPTIONS)

CPU times: user 5.1 ms, sys: 6.33 ms, total: 11.4 ms
Wall time: 1.26 s


In [3]:
# python3 -m spacy download en
nlp = spacy.load('en')

def tokenize(line):
    return [(token.text, token.tag_) for token in nlp(line)]

In [4]:
def numerical(line):
    # replace everything to '' except whitespace, alphanumeric character
    line = re.sub(r'[^\w\s]', '', line)
    token_tag_pairs = tokenize(line)
    for pair in token_tag_pairs:
        # if the word is not numerical
        if not pair[1] == "CD":
            return False
    return True

def nouns_only(line):
    # replace everything to '' except whitespace, alphanumeric character
    line = re.sub(r'[^\w\s]', '', line)
    token_tag_pairs = tokenize(line)
    for pair in token_tag_pairs:
        # if the word is not a noun or cardinal number
        if not (pair[1] == "NN" or pair[1] == "NNS"):
            return False
    return True

In [5]:
type_exceptions = ['can', 'tablespoon', 'clove']

for exception in type_exceptions:
    print(tokenize(exception), spacy.explain(tokenize(exception)[0][1]))

[('can', 'MD')] verb, modal auxiliary
[('tablespoon', 'VB')] verb, base form
[('clove', 'VB')] verb, base form


In [6]:
def extract_quantity_in_backets(line):
    # find '(abc)' where 'abc' is in arbitrary length and 'abc' does not contain brackets
    pattern = re.compile(r'\([^\(\)]*\)') 
    match = re.findall(pattern, line)
    if len(match) != 0:
        # if no numerical value or line_split length > 3 
        if not any(char.isdigit() for char in match[0]) or len(match[0].split()) > 3:
            return None
        return match
    
def extract_preparation(line):
    # find ', abc' where 'abc' is in arbitrary length
    pattern = re.compile(r'[\b]?, [^\(\)]*')
    match = re.findall(pattern, line)
    if len(match) != 0:
        return match
    
def extract_descriptor(ingredient_name):
    type_exceptions = ['parsley', 'garlic', 'chili', 'substitute']
    descriptor = []
    token_tag_pairs = []
    
    for element in ingredient_name.split():
        # treat compound word with hyphen as an adjective
        if '-' in element:
            token_tag_pairs.append((element, 'JJ'))
        else:
            token_tag_pairs.append([(token.text, token.tag_) for token in nlp(element)][0])
    
    for pair in token_tag_pairs:
        # if the word is an adjective, an adverb, or a past participle of a verb, or exception like 'ground'
        if pair[1] == "JJ" or pair[1] == "RB" or pair[1] == "VBN" or pair[0] == 'ground':
            if pair[0] not in type_exceptions:
                descriptor.append(pair[0])
    if len(descriptor) != 0:
        return ' '.join(descriptor)
    
def extract_all(line):
#     print(line)
    type_exceptions = ['can', 'tablespoon', 'oz', 'clove']
    quantity_split = []
    measurement = None
    
    # extract preparation
    line = line.replace(' -', ',')
    preparation = extract_preparation(line)
    if preparation:
#         print('preparation[0]: {0}|'.format(preparation[0]))
        preparation = preparation[0].strip()
        line = re.sub(r'{0}'.format(preparation), '', line)
        # remove ', ' prefix
        preparation = preparation[2:]
        
    # extract quantity in backets
    quantity_in_brackets = extract_quantity_in_backets(line)
    if quantity_in_brackets:
        line = re.sub(r'\({0}\)'.format(quantity_in_brackets[0]), '', line)
        quantity_in_brackets = quantity_in_brackets[0]
    
    line_split = line.split()
    # extract quantity from the first word if the word contains a digit
    if any(char.isdigit() for char in line_split[0]):
        quantity_split.append(line_split[0])
    
        # extract quantity from the second word if the word contains a digit
        if any(char.isdigit() for char in line_split[1]):
            quantity_split.append(line_split[1])
            # check measurement type
            # to avoid case like '1 large tomato, seeded and chopped'
            if nouns_only(line_split[2]) or line_split[2] in type_exceptions:
                measurement = line_split[2]
        else:
            # check line_split length for case like '1 egg' or '1/2 onion, chopped'
            if len(line_split) > 2 and (nouns_only(line_split[1]) or line_split[1] in type_exceptions):
                measurement = line_split[1]
        line = re.sub(r'{0}'.format(' '.join(quantity_split)), '', line)
    
    if measurement:
        line = re.sub(r'{0}'.format(measurement), '', line)
    
    # append quantity in backets at the end
    if quantity_in_brackets:
        quantity_split.append(quantity_in_brackets)
    quantity = ' '.join(quantity_split)

    # extract ingredient name
    line = re.sub(r'[ ]?®', '', line)
    line = re.sub(r'[ ]?™', '', line)
    ingredient_name = line.strip()
    
    # extract descriptor
    descriptor = extract_descriptor(ingredient_name)
    # remove descriptor if not None
    ingredient = ingredient_name.replace(descriptor, '').strip() if descriptor else ingredient_name
    # if ingredient is empty after removing descriptor
    if ingredient == '':
        ingredient = ingredient_name
    # remove ' to taste' in ingredient if any
    ingredient = re.sub(r'(or)? to taste', '', ingredient)
    ingredient = ' '.join(ingredient.split())
    
    # if 'or to taste' or 'or as needed' in preparation
    if preparation is not None and 'or ' in preparation:
        quantity += ' ' + preparation
        preparation = None    
    
    return quantity, measurement, descriptor, ingredient, preparation

In [28]:
# test on tricky cases
# extract_all('4 roma (plum) tomatoes, chopped')
# extract_all('1 1/2 pounds colossal shrimp, EZ-peel type (deveined and shells split down the back)')
# extract_all('2 teaspoons sesame oil, or to taste')
# extract_all('4 pounds Korean-style short ribs (beef chuck flanken, cut 1/3 to 1/2 inch thick across bones)')
# extract_all('1/2 (14 ounce) package vegetarian ground beef (e.g., Gimme Lean TM)')
# extract_all('1 0.42 oz packet concentrated vegetable base (such as Swanson® Vegetable Flavor Boost®) (optional)')
# extract_all('2 avocados - peeled, pitted, and sliced')
extract_all('ground black pepper to taste')
# extract_all('1 (15 ounce) can garbanzo beans (chickpeas), drained and rinsed')

('', None, 'ground black', 'pepper', None)

In [8]:
# test on tricky cases
# extract_descriptor('extra-firm tofu') 
# extract_descriptor('all-purpose flour') 
extract_descriptor('package vegetarian ground beef substitute') 
# extract_descriptor('1/2 cup chopped parsley')
# extract_descriptor('1 1/2 pounds colossal shrimp, EZ-peel type (deveined and shells split down the back)')

'vegetarian ground'

In [9]:
def get_ingredient_list(url):
#     page = requests.get(url)
#     soup = BeautifulSoup(page.content, 'html.parser')
    global driver
    try:
        driver.get(url)
    except:
        print('relaunching webdriver...')
        driver = webdriver.Chrome(options=OPTIONS)
        driver.get(url)
        
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # extract ingredients section from the webpage
    ingredients = set([element.label.text.strip() for element in soup.find_all(class_='checkList__line')])
    # remove unnecessary elements
    unnecessary = ['', 'Add all ingredients to list']
    for i in unnecessary:
        if i in ingredients:
            ingredients.remove(i)
    return ingredients

In [10]:
%%time
# test
get_ingredient_list('https://www.allrecipes.com/recipe/13928/enchilada-casserole/')

CPU times: user 472 ms, sys: 120 ms, total: 592 ms
Wall time: 7.51 s


{'1 (15 ounce) can black beans, rinsed and drained',
 '1 (19 ounce) can enchilada sauce',
 '1 (4 ounce) can diced green chile peppers',
 '1 (6 ounce) can sliced black olives',
 '1 (8 ounce) package tempeh, crumbled',
 '1 jalapeno pepper, seeded and minced',
 '1 onion, chopped',
 '2 cloves garlic, minced',
 '6 (6 inch) corn tortillas',
 '8 ounces shredded Cheddar cheese'}

In [11]:
def lemmatize(line):
    if len(line.split()) == 1:
        return [token.lemma_ for token in nlp(line)][0]
    return line

In [12]:
def get_ingredients_from_url(url):
    print(url)
    ingredients_url = set()
    ingredients_measurement_dict_url = defaultdict(set)
    ingredients_descriptor_dict_url = defaultdict(set)
    ingredients_preparation_dict_url = defaultdict(set)
    
    ingredient_list = get_ingredient_list(url)
    for line in ingredient_list:
        # exceptions like "topping:"
        if ':' in line:
            print(line)
            print(url)
            continue
        
        quantity, measurement, descriptor, ingredient, preparation = extract_all(line)
            
        # lemmatize ingredient
#         ingredient = lemmatize(ingredient)
        
        if ingredient not in ingredients_measurement_dict_url:
            ingredients_measurement_dict_url[ingredient] |= {measurement}
        if ingredient not in ingredients_descriptor_dict_url:
            ingredients_descriptor_dict_url[ingredient] |= {descriptor}
        if ingredient not in ingredients_preparation_dict_url:
            ingredients_preparation_dict_url[ingredient] |= {preparation}
        ingredients_url |= {ingredient}
        
        # test
        print(line)
        print('quantity:', quantity)
        print('measurement:', measurement)
        print('descriptor:', descriptor)
        print('ingredient without descriptor:', ingredient)
        print('preparation:', preparation)
        print()
    
    return ingredients_url, ingredients_measurement_dict_url, ingredients_descriptor_dict_url, ingredients_preparation_dict_url

In [13]:
def get_recipe_urls(url):
    global driver
    try:
        driver.get(url)
    except:
        print('relaunching webdriver...')
        driver = webdriver.Chrome(options=OPTIONS)
        driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
#     page = requests.get(url)
#     soup = BeautifulSoup(page.content, 'html.parser')
    recipe_cards = soup.find_all('div', class_='fixed-recipe-card__info')
    recipe_urls = []
    for recipe_card in recipe_cards:
        url = recipe_card.find('a')['href']
        recipe_urls.append(url)
    return recipe_urls

In [14]:
def get_all_ingredients_in_category(category_url):
    ingredients_freq_dict_all = dict()
    ingredients_measurement_dict_all = defaultdict(set)
    ingredients_descriptor_dict_all = defaultdict(set)
    ingredients_preparation_dict_all = defaultdict(set)
    
    recipe_urls = get_recipe_urls(category_url)
    for recipe_url in recipe_urls:
        recipe_ingredients, recipe_ingredients_measurement_dict, ingredients_descriptor_dict, ingredients_preparation_dict = get_ingredients_from_url(recipe_url)
        for i in recipe_ingredients:
            if i in ingredients_freq_dict_all:
                ingredients_freq_dict_all[i] += 1
            else:
                ingredients_freq_dict_all[i] = 1
                
        for i in recipe_ingredients:
            if i in ingredients_measurement_dict_all:
                ingredients_measurement_dict_all[i] |= recipe_ingredients_measurement_dict[i]
            else:
                ingredients_measurement_dict_all[i] = recipe_ingredients_measurement_dict[i]
    
        for i in recipe_ingredients:
            if i in ingredients_descriptor_dict_all:
                ingredients_descriptor_dict_all[i] |= ingredients_descriptor_dict[i]
            else:
                ingredients_descriptor_dict_all[i] = ingredients_descriptor_dict[i]
                
        for i in recipe_ingredients:
            if i in ingredients_preparation_dict_all:
                ingredients_preparation_dict_all[i] |= ingredients_preparation_dict[i]
            else:
                ingredients_preparation_dict_all[i] = ingredients_preparation_dict[i]  
                
        # scraping pause to avoid being banned by site
#         time.sleep(1)
    return ingredients_freq_dict_all, ingredients_measurement_dict_all, ingredients_descriptor_dict_all, ingredients_preparation_dict_all

In [15]:
# 4 pages of vegetarian protein recipes
# 6 pages of meat recipes
# 2 pages of seafood recipes

In [16]:
%%time
vegetarian_protein_ingredients_freq_dict = dict()
vegetarian_protein_ingredients_measurement_dict = defaultdict(set)
vegetarian_protein_ingredients_descriptor_dict = defaultdict(set)
vegetarian_protein_ingredients_preparation_dict = defaultdict(set)
    
for page in range(4):
    vegetarian_protein_url = 'https://www.allrecipes.com/recipes/16778/everyday-cooking/vegetarian/protein/?page=' + str(page+1)
    ingredients_cat, ingredients_measurement_dict_cat, ingredients_descriptor_dict_cat, ingredients_preparation_dict_cat = get_all_ingredients_in_category(vegetarian_protein_url)
    for i in ingredients_cat.keys():
        if i in vegetarian_protein_ingredients_freq_dict:
            vegetarian_protein_ingredients_freq_dict[i] += ingredients_cat[i]
        else:
            vegetarian_protein_ingredients_freq_dict[i] = ingredients_cat[i]
    
    for i in ingredients_measurement_dict_cat.keys():
        if i in vegetarian_protein_ingredients_measurement_dict:
            vegetarian_protein_ingredients_measurement_dict[i] |= ingredients_measurement_dict_cat[i]
        else:
            vegetarian_protein_ingredients_measurement_dict[i] = ingredients_measurement_dict_cat[i]
    
    for i in ingredients_descriptor_dict_cat.keys():
        if i in vegetarian_protein_ingredients_descriptor_dict:
            vegetarian_protein_ingredients_descriptor_dict[i] |= ingredients_descriptor_dict_cat[i]
        else:
            vegetarian_protein_ingredients_descriptor_dict[i] = ingredients_descriptor_dict_cat[i]
    
    for i in ingredients_preparation_dict_cat.keys():
        if i in vegetarian_protein_ingredients_preparation_dict:
            vegetarian_protein_ingredients_preparation_dict[i] |= ingredients_preparation_dict_cat[i]
        else:
            vegetarian_protein_ingredients_preparation_dict[i] = ingredients_preparation_dict_cat[i]

https://www.allrecipes.com/recipe/13886/stuffed-bell-peppers/?internalSource=rotd&referringId=16778&referringContentType=Recipe Hub
4 tablespoons olive oil
quantity: 4
measurement: tablespoons
descriptor: None
ingredient without descriptor: olive oil
preparation: None

2 cups tomato sauce
quantity: 2
measurement: cups
descriptor: None
ingredient without descriptor: tomato sauce
preparation: None

1 onion, chopped
quantity: 1
measurement: None
descriptor: None
ingredient without descriptor: onion
preparation: chopped

2 tablespoons chopped fresh parsley
quantity: 2
measurement: tablespoons
descriptor: chopped fresh
ingredient without descriptor: parsley
preparation: None

1/2 cup uncooked white rice
quantity: 1/2
measurement: cup
descriptor: uncooked white
ingredient without descriptor: rice
preparation: None

salt to taste
quantity: 
measurement: None
descriptor: None
ingredient without descriptor: salt
preparation: None

4 ounces shredded mozzarella cheese
quantity: 4
measurement: oun

1 tablespoon sesame seeds
quantity: 1
measurement: tablespoon
descriptor: None
ingredient without descriptor: sesame seeds
preparation: None

https://www.allrecipes.com/recipe/46960/barbeque-tempeh-sandwiches/?internalSource=staff pick&referringId=16778&referringContentType=Recipe Hub
1 red bell pepper, seeded and chopped
quantity: 1
measurement: None
descriptor: red
ingredient without descriptor: bell pepper
preparation: seeded and chopped

4 kaiser rolls, split and toasted
quantity: 4
measurement: kaiser
descriptor: None
ingredient without descriptor: rolls
preparation: split and toasted

1 cup barbecue sauce, your choice
quantity: 1
measurement: cup
descriptor: None
ingredient without descriptor: barbecue sauce
preparation: your choice

1 green bell pepper, seeded and chopped
quantity: 1
measurement: None
descriptor: green
ingredient without descriptor: bell pepper
preparation: seeded and chopped

1 medium onion, chopped
quantity: 1
measurement: None
descriptor: medium
ingredient wi

2 bunches green onions
quantity: 2
measurement: bunches
descriptor: green
ingredient without descriptor: onions
preparation: None

1 (14 ounce) can light coconut milk
quantity: 1 (14 ounce)
measurement: can
descriptor: light
ingredient without descriptor: coconut milk
preparation: None

1 pound firm tofu, cut into 3/4 inch cubes
quantity: 1
measurement: pound
descriptor: firm
ingredient without descriptor: tofu
preparation: cut into 3/4 inch cubes

2 teaspoons chile paste
quantity: 2
measurement: teaspoons
descriptor: chile
ingredient without descriptor: paste
preparation: None

https://www.allrecipes.com/recipe/86313/baked-tofu-bites/?internalSource=hub recipe&referringId=16778&referringContentType=Recipe Hub
2 tablespoons ketchup
quantity: 2
measurement: tablespoons
descriptor: None
ingredient without descriptor: ketchup
preparation: None

1/4 cup soy sauce
quantity: 1/4
measurement: cup
descriptor: None
ingredient without descriptor: soy sauce
preparation: None

1 (16 ounce) package

1/4 cup chopped fresh cilantro
quantity: 1/4
measurement: cup
descriptor: chopped fresh
ingredient without descriptor: cilantro
preparation: None

https://www.allrecipes.com/recipe/25345/tofu-lasagna/?internalSource=hub recipe&referringId=16778&referringContentType=Recipe Hub
1/4 teaspoon salt
quantity: 1/4
measurement: teaspoon
descriptor: None
ingredient without descriptor: salt
preparation: None

1/2 (12 ounce) package uncooked lasagna noodles
quantity: 1/2 (12 ounce)
measurement: package
descriptor: uncooked
ingredient without descriptor: lasagna noodles
preparation: None

1 cup spaghetti sauce
quantity: 1
measurement: cup
descriptor: None
ingredient without descriptor: spaghetti sauce
preparation: None

1 tablespoon dried parsley
quantity: 1
measurement: tablespoon
descriptor: dried
ingredient without descriptor: parsley
preparation: None

2 cups shredded mozzarella cheese, divided
quantity: 2
measurement: cups
descriptor: shredded
ingredient without descriptor: mozzarella cheese


1 medium onions, minced
quantity: 1
measurement: None
descriptor: medium
ingredient without descriptor: onions
preparation: minced

1 teaspoon salt
quantity: 1
measurement: teaspoon
descriptor: None
ingredient without descriptor: salt
preparation: None

1 (14 ounce) package extra firm tofu, diced
quantity: 1 (14 ounce)
measurement: package
descriptor: extra firm
ingredient without descriptor: tofu
preparation: diced

1 1/2 cups frozen peas, thawed
quantity: 1 1/2
measurement: cups
descriptor: frozen
ingredient without descriptor: peas
preparation: thawed

1 tablespoon red pepper paste
quantity: 1
measurement: tablespoon
descriptor: red
ingredient without descriptor: pepper paste
preparation: None

1 (16 ounce) can coconut milk
quantity: 1 (16 ounce)
measurement: can
descriptor: None
ingredient without descriptor: coconut milk
preparation: None

1 teaspoon curry powder
quantity: 1
measurement: teaspoon
descriptor: None
ingredient without descriptor: curry powder
preparation: None

1 tab

...

1 cup frozen peas
quantity: 1
measurement: cup
descriptor: frozen
ingredient without descriptor: peas
preparation: None

1 teaspoon garam masala
quantity: 1
measurement: teaspoon
descriptor: None
ingredient without descriptor: garam masala
preparation: None

1 (16 ounce) can tomato sauce
quantity: 1 (16 ounce)
measurement: can
descriptor: None
ingredient without descriptor: tomato sauce
preparation: None

4 teaspoons ground coriander
quantity: 4
measurement: teaspoons
descriptor: ground
ingredient without descriptor: coriander
preparation: None

2 tablespoons unsalted butter
quantity: 2
measurement: tablespoons
descriptor: unsalted
ingredient without descriptor: butter
preparation: None

2 teaspoons ground cumin
quantity: 2
measurement: teaspoons
descriptor: ground
ingredient without descriptor: cumin
preparation: None

4 cloves garlic, minced
quantity: 4
measurement: cloves
descriptor: None
ingredient without descriptor: garlic
preparation: minced

1/2 cup plain yogurt
quantity: 1/2
m

1 pinch salt
quantity: 1
measurement: pinch
descriptor: None
ingredient without descriptor: salt
preparation: None

1 (12 ounce) package silken tofu, undrained
quantity: 1 (12 ounce)
measurement: package
descriptor: silken
ingredient without descriptor: tofu
preparation: undrained

1 cup cocoa powder
quantity: 1
measurement: cup
descriptor: None
ingredient without descriptor: cocoa powder
preparation: None

2 1/3 cups turbinado sugar
quantity: 2 1/3
measurement: cups
descriptor: None
ingredient without descriptor: turbinado sugar
preparation: None

2 tablespoons safflower oil
quantity: 2
measurement: tablespoons
descriptor: safflower
ingredient without descriptor: oil
preparation: None

1/3 cup instant decaffeinated coffee powder
quantity: 1/3
measurement: cup
descriptor: instant decaffeinated
ingredient without descriptor: coffee powder
preparation: None

1 teaspoon vanilla extract
quantity: 1
measurement: teaspoon
descriptor: None
ingredient without descriptor: vanilla extract
prepar

1 egg, beaten
quantity: 1
measurement: None
descriptor: None
ingredient without descriptor: egg
preparation: beaten

salt and pepper to taste
quantity: 
measurement: None
descriptor: None
ingredient without descriptor: salt and pepper
preparation: None

1 (12 ounce) package firm tofu
quantity: 1 (12 ounce)
measurement: package
descriptor: firm
ingredient without descriptor: tofu
preparation: None

2 teaspoons vegetable oil
quantity: 2
measurement: teaspoons
descriptor: vegetable
ingredient without descriptor: oil
preparation: None

1/2 cup vegetable oil for frying
quantity: 1/2
measurement: cup
descriptor: vegetable
ingredient without descriptor: oil for frying
preparation: None

1 small onion, chopped
quantity: 1
measurement: None
descriptor: small
ingredient without descriptor: onion
preparation: chopped

1 celery, chopped
quantity: 1
measurement: None
descriptor: None
ingredient without descriptor: celery
preparation: chopped

1/4 cup shredded Cheddar cheese
quantity: 1/4
measuremen

1 bay leaf
quantity: 1
measurement: bay
descriptor: None
ingredient without descriptor: leaf
preparation: None

1 large red onion, chopped
quantity: 1
measurement: None
descriptor: large red
ingredient without descriptor: onion
preparation: chopped

1/2 teaspoon black pepper
quantity: 1/2
measurement: teaspoon
descriptor: black
ingredient without descriptor: pepper
preparation: None

2 tablespoons nutritional yeast (optional)
quantity: 2
measurement: tablespoons
descriptor: nutritional
ingredient without descriptor: yeast (optional)
preparation: None

3/4 cup dry lentils
quantity: 3/4
measurement: cup
descriptor: dry
ingredient without descriptor: lentils
preparation: None

1 (12 ounce) package rotini pasta
quantity: 1 (12 ounce)
measurement: package
descriptor: None
ingredient without descriptor: rotini pasta
preparation: None

1 bunch kale, stems removed and leaves coarsely chopped
quantity: 1
measurement: bunch
descriptor: None
ingredient without descriptor: kale
preparation: stems 

3 fresh green chile peppers
quantity: 3
measurement: None
descriptor: fresh green chile
ingredient without descriptor: peppers
preparation: None

4 cloves garlic, peeled
quantity: 4
measurement: cloves
descriptor: None
ingredient without descriptor: garlic
preparation: peeled

2 tablespoons lime juice
quantity: 2
measurement: tablespoons
descriptor: None
ingredient without descriptor: lime juice
preparation: None

3 teaspoons light brown sugar
quantity: 3
measurement: teaspoons
descriptor: light brown
ingredient without descriptor: sugar
preparation: None

1/2 cup chopped fresh basil
quantity: 1/2
measurement: cup
descriptor: chopped fresh
ingredient without descriptor: basil
preparation: None

1 tablespoon vegetable oil
quantity: 1
measurement: tablespoon
descriptor: vegetable
ingredient without descriptor: oil
preparation: None

1 small onion, quartered
quantity: 1
measurement: None
descriptor: small
ingredient without descriptor: onion
preparation: quartered

5 small eggplants
quant

In [17]:
sorted(vegetarian_protein_ingredients_freq_dict.items(), key=lambda pair: pair[1], reverse=True)

[('tofu', 65),
 ('onion', 39),
 ('garlic', 37),
 ('salt', 33),
 ('pepper', 30),
 ('oil', 27),
 ('soy sauce', 22),
 ('water', 20),
 ('olive oil', 18),
 ('bell pepper', 17),
 ('mushrooms', 14),
 ('onions', 14),
 ('sugar', 12),
 ('cilantro', 11),
 ('sesame oil', 10),
 ('garlic powder', 10),
 ('tomato sauce', 9),
 ('cumin', 9),
 ('carrots', 9),
 ('tomatoes', 9),
 ('celery', 8),
 ('tempeh', 8),
 ('broth', 8),
 ('basil', 8),
 ('coconut milk', 7),
 ('salt and pepper', 7),
 ('ginger root', 7),
 ('mozzarella cheese', 6),
 ('spinach', 6),
 ('oregano', 6),
 ('curry powder', 6),
 ('cayenne pepper', 6),
 ('chili powder', 6),
 ('beans', 6),
 ('butter', 6),
 ('parsley', 5),
 ('rice', 5),
 ('tamari', 5),
 ('soy milk', 5),
 ('ginger', 5),
 ('barbecue sauce', 5),
 ('Cheddar cheese', 5),
 ('lime juice', 5),
 ('peppers', 5),
 ('vinegar', 5),
 ('cornstarch', 5),
 ('honey', 5),
 ('vanilla extract', 5),
 ('zucchini', 4),
 ('yeast', 4),
 ('coriander', 4),
 ('potatoes', 4),
 ('peas', 4),
 ('rice vinegar', 4),


In [18]:
vegetarian_protein_ingredients_preparation_dict

defaultdict(set,
            {'parsley': {None},
             'bell peppers': {None, 'halved and seeded'},
             'onion': {None,
              'chopped',
              'coarsely chopped',
              'finely chopped',
              'finely diced',
              'minced',
              'quartered',
              'sliced',
              'thinly sliced'},
             'mozzarella cheese': {None, 'divided'},
             'olive oil': {None, 'as needed'},
             'pepper': {None, 'chopped', 'diced', 'seeded and minced'},
             'water': {None},
             'protein': {None},
             'rice': {None},
             'salt': {None},
             'tomato sauce': {None},
             'tofu': {None,
              'crumbled',
              'cubed',
              'cut into 1-inch cubes',
              'cut into 1/4-inch cubes',
              'cut into 3/4 inch cubes',
              'cut into bite-size pieces',
              'cut into bite-sized cubes',
              'diced',


In [19]:
%%time
meat_ingredients_freq_dict = dict()
meat_ingredients_measurement_dict = defaultdict(set)
meat_ingredients_descriptor_dict = defaultdict(set)
meat_ingredients_preparation_dict = defaultdict(set)

for index in range(200, 206):
    meat_url = 'https://www.allrecipes.com/recipes/{}/meat-and-poultry/'.format(index)
    ingredients_cat, ingredients_measurement_dict_cat, ingredients_descriptor_dict_cat, ingredients_preparation_dict_cat = get_all_ingredients_in_category(meat_url)
    for i in ingredients_cat.keys():
        if i in meat_ingredients_freq_dict:
            meat_ingredients_freq_dict[i] += ingredients_cat[i]
        else:
            meat_ingredients_freq_dict[i] = ingredients_cat[i]
    
    for i in ingredients_measurement_dict_cat.keys():
        if i in meat_ingredients_measurement_dict:
            meat_ingredients_measurement_dict[i] |= ingredients_measurement_dict_cat[i]
        else:
            meat_ingredients_measurement_dict[i] = ingredients_measurement_dict_cat[i]
    
    for i in ingredients_descriptor_dict_cat.keys():
        if i in meat_ingredients_descriptor_dict:
            meat_ingredients_descriptor_dict[i] |= ingredients_descriptor_dict_cat[i]
        else:
            meat_ingredients_descriptor_dict[i] = ingredients_descriptor_dict_cat[i]
    
    for i in ingredients_preparation_dict_cat.keys():
        if i in meat_ingredients_preparation_dict:
            meat_ingredients_preparation_dict[i] |= ingredients_preparation_dict_cat[i]
        else:
            meat_ingredients_preparation_dict[i] = ingredients_preparation_dict_cat[i]

https://www.allrecipes.com/recipe/15825/cajun-style-meatloaf/?internalSource=rotd&referringId=200&referringContentType=Recipe Hub
1 teaspoon salt
quantity: 1
measurement: teaspoon
descriptor: None
ingredient without descriptor: salt
preparation: None

2 bay leaves
quantity: 2
measurement: bay
descriptor: None
ingredient without descriptor: leaves
preparation: None

1 1/2 pounds ground beef
quantity: 1 1/2
measurement: pounds
descriptor: ground
ingredient without descriptor: beef
preparation: None

1/2 pound andouille sausage, casings removed
quantity: 1/2
measurement: pound
descriptor: None
ingredient without descriptor: andouille sausage
preparation: casings removed

1/2 teaspoon ground nutmeg
quantity: 1/2
measurement: teaspoon
descriptor: ground nutmeg
ingredient without descriptor: ground nutmeg
preparation: None

1/2 teaspoon ground cumin
quantity: 1/2
measurement: teaspoon
descriptor: ground
ingredient without descriptor: cumin
preparation: None

1/2 teaspoon ground white pepper


2 tablespoons minced garlic
quantity: 2
measurement: tablespoons
descriptor: minced
ingredient without descriptor: garlic
preparation: None

https://www.allrecipes.com/recipe/255660/abuelas-picadillo/?internalSource=staff pick&referringId=200&referringContentType=Recipe Hub
2 pounds lean ground beef
quantity: 2
measurement: pounds
descriptor: lean ground
ingredient without descriptor: beef
preparation: None

1/2 green bell pepper, chopped
quantity: 1/2
measurement: None
descriptor: green
ingredient without descriptor: bell pepper
preparation: chopped

1 teaspoon salt, or to taste
quantity: 1 or to taste
measurement: teaspoon
descriptor: None
ingredient without descriptor: salt
preparation: None

1/2 teaspoon garlic powder
quantity: 1/2
measurement: teaspoon
descriptor: None
ingredient without descriptor: garlic powder
preparation: None

1/2 teaspoon ground cumin
quantity: 1/2
measurement: teaspoon
descriptor: ground
ingredient without descriptor: cumin
preparation: None

1/2 onion, cho

1 cup frozen green peas
quantity: 1
measurement: cup
descriptor: frozen green
ingredient without descriptor: peas
preparation: None

https://www.allrecipes.com/recipe/160399/prime-rib-au-jus-with-yorkshire-pudding/?internalSource=staff pick&referringId=200&referringContentType=Recipe Hub
1 pinch salt
quantity: 1
measurement: pinch
descriptor: None
ingredient without descriptor: salt
preparation: None

1/2 cup white wine
quantity: 1/2
measurement: cup
descriptor: white
ingredient without descriptor: wine
preparation: None

2 cups milk
quantity: 2
measurement: cups
descriptor: None
ingredient without descriptor: milk
preparation: None

1 teaspoon freshly ground black pepper
quantity: 1
measurement: teaspoon
descriptor: freshly ground black
ingredient without descriptor: pepper
preparation: None

1 (5 pound) bone-in beef rib roast
quantity: 1 (5 pound)
measurement: bone-in
descriptor: None
ingredient without descriptor: beef rib roast
preparation: None

2 cups eggs
quantity: 2
measurement

1 pound lean ground beef
quantity: 1
measurement: pound
descriptor: lean ground
ingredient without descriptor: beef
preparation: None

6 ounces mozzarella cheese, shredded
quantity: 6
measurement: ounces
descriptor: None
ingredient without descriptor: mozzarella cheese
preparation: shredded

1 onion, chopped
quantity: 1
measurement: None
descriptor: None
ingredient without descriptor: onion
preparation: chopped

2 tablespoons grated Parmesan cheese
quantity: 2
measurement: tablespoons
descriptor: grated
ingredient without descriptor: Parmesan cheese
preparation: None

2 (26 ounce) jars spaghetti sauce
quantity: 2 (26 ounce)
measurement: jars
descriptor: None
ingredient without descriptor: spaghetti sauce
preparation: None

6 ounces provolone cheese, sliced
quantity: 6
measurement: ounces
descriptor: None
ingredient without descriptor: provolone cheese
preparation: sliced

1 1/2 cups sour cream
quantity: 1 1/2
measurement: cups
descriptor: sour
ingredient without descriptor: cream
prepa

1 teaspoon Worcestershire sauce
quantity: 1
measurement: teaspoon
descriptor: None
ingredient without descriptor: Worcestershire sauce
preparation: None

1 teaspoon paprika
quantity: 1
measurement: teaspoon
descriptor: None
ingredient without descriptor: paprika
preparation: None

https://www.allrecipes.com/recipe/14595/salisbury-steak/?internalSource=hub recipe&referringId=200&referringContentType=Recipe Hub
1/4 teaspoon salt
quantity: 1/4
measurement: teaspoon
descriptor: None
ingredient without descriptor: salt
preparation: None

1/4 cup ketchup
quantity: 1/4
measurement: cup
descriptor: None
ingredient without descriptor: ketchup
preparation: None

1 tablespoon Worcestershire sauce
quantity: 1
measurement: tablespoon
descriptor: None
ingredient without descriptor: Worcestershire sauce
preparation: None

1/4 cup water
quantity: 1/4
measurement: cup
descriptor: None
ingredient without descriptor: water
preparation: None

1/2 cup dry bread crumbs
quantity: 1/2
measurement: cup
descrip

...

1/2 pound fresh mushrooms, sliced
quantity: 1/2
measurement: pound
descriptor: fresh
ingredient without descriptor: mushrooms
preparation: sliced

1 onion, chopped
quantity: 1
measurement: None
descriptor: None
ingredient without descriptor: onion
preparation: chopped

salt and pepper to taste
quantity: 
measurement: None
descriptor: None
ingredient without descriptor: salt and pepper
preparation: None

1 pinch garlic salt, or to taste
quantity: 1 or to taste
measurement: pinch
descriptor: None
ingredient without descriptor: garlic salt
preparation: None

1 (10.75 ounce) can condensed cream of mushroom soup
quantity: 1 (10.75 ounce)
measurement: can
descriptor: condensed
ingredient without descriptor: cream of mushroom soup
preparation: None

4 pork chops
quantity: 4
measurement: pork
descriptor: None
ingredient without descriptor: chops
preparation: None

https://www.allrecipes.com/recipe/235378/super-easy-pulled-pork-sandwiches/?internalSource=staff pick&referringId=205&referringCont

1/4 teaspoon ground black pepper
quantity: 1/4
measurement: teaspoon
descriptor: ground black
ingredient without descriptor: pepper
preparation: None

1/2 teaspoon fennel seeds
quantity: 1/2
measurement: teaspoon
descriptor: None
ingredient without descriptor: fennel seeds
preparation: None

1/2 teaspoon salt
quantity: 1/2
measurement: teaspoon
descriptor: None
ingredient without descriptor: salt
preparation: None

4 tablespoons chopped fresh parsley
quantity: 4
measurement: tablespoons
descriptor: chopped fresh
ingredient without descriptor: parsley
preparation: None

3/4 cup grated Parmesan cheese
quantity: 3/4
measurement: cup
descriptor: grated
ingredient without descriptor: Parmesan cheese
preparation: None

3/4 pound mozzarella cheese, sliced
quantity: 3/4
measurement: pound
descriptor: None
ingredient without descriptor: mozzarella cheese
preparation: sliced

1 egg
quantity: 1
measurement: None
descriptor: None
ingredient without descriptor: egg
preparation: None

2 cloves garli

8 hamburger buns, split and lightly toasted
quantity: 8
measurement: hamburger
descriptor: None
ingredient without descriptor: buns
preparation: split and lightly toasted

1 (2 pound) pork tenderloin
quantity: 1 (2 pound)
measurement: pork
descriptor: None
ingredient without descriptor: tenderloin
preparation: None

1 (18 ounce) bottle your favorite barbecue sauce
quantity: 1 (18 ounce)
measurement: bottle
descriptor: favorite
ingredient without descriptor: your barbecue sauce
preparation: None

1 (12 fluid ounce) can or bottle root beer
quantity: 1 (12 fluid ounce)
measurement: can
descriptor: None
ingredient without descriptor: or bottle root beer
preparation: None

https://www.allrecipes.com/recipe/26257/bow-ties-with-sausage-tomatoes-and-cream/?internalSource=hub recipe&referringId=205&referringContentType=Recipe Hub
3 tablespoons minced fresh parsley
quantity: 3
measurement: tablespoons
descriptor: minced fresh
ingredient without descriptor: parsley
preparation: None

1 1/2 cups h

1 large onion, chopped
quantity: 1
measurement: None
descriptor: large
ingredient without descriptor: onion
preparation: chopped

1 pound skinless, boneless chicken breast halves - cut into 1 inch cubes
quantity: 1
measurement: pound
descriptor: None
ingredient without descriptor: skinless
preparation: boneless chicken breast halves, cut into 1 inch cubes

1 pound andouille sausage, sliced
quantity: 1
measurement: pound
descriptor: None
ingredient without descriptor: andouille sausage
preparation: sliced

1 cup chicken broth
quantity: 1
measurement: cup
descriptor: None
ingredient without descriptor: chicken broth
preparation: None

2 teaspoons Cajun seasoning
quantity: 2
measurement: teaspoons
descriptor: None
ingredient without descriptor: Cajun seasoning
preparation: None

1/2 teaspoon dried thyme
quantity: 1/2
measurement: teaspoon
descriptor: dried
ingredient without descriptor: thyme
preparation: None

1 cup chopped celery
quantity: 1
measurement: cup
descriptor: chopped
ingredie

1 cup thinly sliced carrots
quantity: 1
measurement: cup
descriptor: thinly sliced
ingredient without descriptor: carrots
preparation: None

1/2 teaspoon dried oregano
quantity: 1/2
measurement: teaspoon
descriptor: dried
ingredient without descriptor: oregano
preparation: None

1 cup chopped onion
quantity: 1
measurement: cup
descriptor: chopped
ingredient without descriptor: onion
preparation: None

8 ounces fresh tortellini pasta
quantity: 8
measurement: ounces
descriptor: fresh
ingredient without descriptor: tortellini pasta
preparation: None

https://www.allrecipes.com/recipe/132815/balsamic-roasted-pork-loin/?internalSource=hub recipe&referringId=205&referringContentType=Recipe Hub
1/2 cup balsamic vinegar
quantity: 1/2
measurement: cup
descriptor: balsamic
ingredient without descriptor: vinegar
preparation: None

1/2 cup olive oil
quantity: 1/2
measurement: cup
descriptor: None
ingredient without descriptor: olive oil
preparation: None

2 tablespoons steak seasoning rub
quantity

In [20]:
sorted(meat_ingredients_freq_dict.items(), key=lambda pair: pair[1], reverse=True);

In [21]:
meat_ingredients_preparation_dict;

In [22]:
%%time
seafood_ingredients_freq_dict = dict()
seafood_ingredients_measurement_dict = defaultdict(set)
seafood_ingredients_descriptor_dict = defaultdict(set)
seafood_ingredients_preparation_dict = defaultdict(set)

for page in range(2):
    seafood_url = 'https://www.allrecipes.com/recipes/93/seafood/' + str(page+1)
    ingredients_cat, ingredients_measurement_dict_cat, ingredients_descriptor_dict_cat, ingredients_preparation_dict_cat = get_all_ingredients_in_category(seafood_url)
    for i in ingredients_cat.keys():
        if i in seafood_ingredients_freq_dict:
            seafood_ingredients_freq_dict[i] += ingredients_cat[i]
        else:
            seafood_ingredients_freq_dict[i] = ingredients_cat[i]
    
    for i in ingredients_measurement_dict_cat.keys():
        if i in seafood_ingredients_measurement_dict:
            seafood_ingredients_measurement_dict[i] |= ingredients_measurement_dict_cat[i]
        else:
            seafood_ingredients_measurement_dict[i] = ingredients_measurement_dict_cat[i]
    
    for i in ingredients_descriptor_dict_cat.keys():
        if i in seafood_ingredients_descriptor_dict:
            seafood_ingredients_descriptor_dict[i] |= ingredients_descriptor_dict_cat[i]
        else:
            seafood_ingredients_descriptor_dict[i] = ingredients_descriptor_dict_cat[i]
    
    for i in ingredients_preparation_dict_cat.keys():
        if i in seafood_ingredients_preparation_dict:
            seafood_ingredients_preparation_dict[i] |= ingredients_preparation_dict_cat[i]
        else:
            seafood_ingredients_preparation_dict[i] = ingredients_preparation_dict_cat[i]

https://www.allrecipes.com/recipe/43719/baked-halibut-steaks/?internalSource=rotd&referringId=93&referringContentType=Recipe Hub
1/2 cup minced onion
quantity: 1/2
measurement: cup
descriptor: minced
ingredient without descriptor: onion
preparation: None

1/4 teaspoon salt
quantity: 1/4
measurement: teaspoon
descriptor: None
ingredient without descriptor: salt
preparation: None

1 clove garlic, peeled and minced
quantity: 1
measurement: clove
descriptor: None
ingredient without descriptor: garlic
preparation: peeled and minced

1 teaspoon olive oil
quantity: 1
measurement: teaspoon
descriptor: None
ingredient without descriptor: olive oil
preparation: None

2 cups diced fresh tomatoes
quantity: 2
measurement: cups
descriptor: diced fresh
ingredient without descriptor: tomatoes
preparation: None

2 tablespoons chopped fresh basil
quantity: 2
measurement: tablespoons
descriptor: chopped fresh
ingredient without descriptor: basil
preparation: None

1 cup diced zucchini
quantity: 1
measure

8 ounces Monterey Jack cheese, shredded
quantity: 8
measurement: ounces
descriptor: None
ingredient without descriptor: Monterey Jack cheese
preparation: shredded

1/2 cup milk
quantity: 1/2
measurement: cup
descriptor: None
ingredient without descriptor: milk
preparation: None

1 onion, chopped
quantity: 1
measurement: None
descriptor: None
ingredient without descriptor: onion
preparation: chopped

2 cloves garlic
quantity: 2
measurement: cloves
descriptor: None
ingredient without descriptor: garlic
preparation: None

8 ounces fontina cheese, shredded
quantity: 8
measurement: ounces
descriptor: None
ingredient without descriptor: fontina cheese
preparation: shredded

seasoned salt to taste
quantity: 
measurement: None
descriptor: seasoned
ingredient without descriptor: salt
preparation: None

1 clove garlic, crushed
quantity: 1
measurement: clove
descriptor: None
ingredient without descriptor: garlic
preparation: crushed

1/2 cup butter
quantity: 1/2
measurement: cup
descriptor: None


3 cloves garlic, minced
quantity: 3
measurement: cloves
descriptor: None
ingredient without descriptor: garlic
preparation: minced

2 tablespoons chopped fresh basil
quantity: 2
measurement: tablespoons
descriptor: chopped fresh
ingredient without descriptor: basil
preparation: None

1/4 teaspoon cayenne pepper
quantity: 1/4
measurement: teaspoon
descriptor: None
ingredient without descriptor: cayenne pepper
preparation: None

1/2 teaspoon salt
quantity: 1/2
measurement: teaspoon
descriptor: None
ingredient without descriptor: salt
preparation: None

1/3 cup olive oil
quantity: 1/3
measurement: cup
descriptor: None
ingredient without descriptor: olive oil
preparation: None

skewers
quantity: 
measurement: None
descriptor: None
ingredient without descriptor: skewers
preparation: None

2 pounds fresh shrimp, peeled and deveined
quantity: 2
measurement: pounds
descriptor: fresh
ingredient without descriptor: shrimp
preparation: peeled and deveined

2 tablespoons red wine vinegar
quantity:

1 teaspoon minced capers
quantity: 1
measurement: teaspoon
descriptor: minced
ingredient without descriptor: capers
preparation: None

1 (12 ounce) package corn tortillas
quantity: 1 (12 ounce)
measurement: package
descriptor: None
ingredient without descriptor: corn tortillas
preparation: None

1/2 medium head cabbage, finely shredded
quantity: 1/2
measurement: None
descriptor: medium
ingredient without descriptor: head cabbage
preparation: finely shredded

1 cup beer
quantity: 1
measurement: cup
descriptor: None
ingredient without descriptor: beer
preparation: None

1 teaspoon baking powder
quantity: 1
measurement: teaspoon
descriptor: None
ingredient without descriptor: baking powder
preparation: None

1/2 teaspoon dried oregano
quantity: 1/2
measurement: teaspoon
descriptor: dried
ingredient without descriptor: oregano
preparation: None

1 quart oil for frying
quantity: 1
measurement: quart
descriptor: None
ingredient without descriptor: oil for frying
preparation: None

1/2 teaspo

1 (15 ounce) jar Alfredo sauce
quantity: 1 (15 ounce)
measurement: jar
descriptor: None
ingredient without descriptor: Alfredo sauce
preparation: None

1 red bell pepper, diced
quantity: 1
measurement: None
descriptor: red
ingredient without descriptor: bell pepper
preparation: diced

https://www.allrecipes.com/recipe/98579/barbies-tuna-salad/?internalSource=hub recipe&referringId=93&referringContentType=Recipe Hub
1 tablespoon dried parsley
quantity: 1
measurement: tablespoon
descriptor: dried
ingredient without descriptor: parsley
preparation: None

1/4 teaspoon curry powder
quantity: 1/4
measurement: teaspoon
descriptor: None
ingredient without descriptor: curry powder
preparation: None

1 (7 ounce) can white tuna, drained and flaked
quantity: 1 (7 ounce)
measurement: can
descriptor: white
ingredient without descriptor: tuna
preparation: drained and flaked

1/8 teaspoon dried minced onion flakes
quantity: 1/8
measurement: teaspoon
descriptor: dried minced
ingredient without descript

1/2 teaspoon paprika
quantity: 1/2
measurement: teaspoon
descriptor: None
ingredient without descriptor: paprika
preparation: None

6 tablespoons unsalted butter, softened
quantity: 6
measurement: tablespoons
descriptor: unsalted
ingredient without descriptor: butter
preparation: softened

1 tablespoon minced garlic
quantity: 1
measurement: tablespoon
descriptor: minced
ingredient without descriptor: garlic
preparation: None

1 tablespoon minced shallots
quantity: 1
measurement: tablespoon
descriptor: minced
ingredient without descriptor: shallots
preparation: None

2 pounds large shrimp - peeled and deveined
quantity: 2
measurement: pounds
descriptor: large
ingredient without descriptor: shrimp
preparation: peeled and deveined

2 tablespoons minced fresh chives
quantity: 2
measurement: tablespoons
descriptor: minced fresh
ingredient without descriptor: chives
preparation: None

salt and freshly ground black pepper to taste
quantity: 
measurement: None
descriptor: freshly ground black


relaunching webdriver...
4 (6 ounce) tuna steaks
quantity: 4 (6 ounce)
measurement: tuna
descriptor: None
ingredient without descriptor: steaks
preparation: None

1/4 cup soy sauce
quantity: 1/4
measurement: cup
descriptor: None
ingredient without descriptor: soy sauce
preparation: None

1/2 cup sesame seeds
quantity: 1/2
measurement: cup
descriptor: None
ingredient without descriptor: sesame seeds
preparation: None

2 tablespoons sesame oil
quantity: 2
measurement: tablespoons
descriptor: None
ingredient without descriptor: sesame oil
preparation: None

1 tablespoon mirin (Japanese sweet wine)
quantity: 1
measurement: tablespoon
descriptor: sweet
ingredient without descriptor: mirin (Japanese wine)
preparation: None

1 tablespoon honey
quantity: 1
measurement: tablespoon
descriptor: None
ingredient without descriptor: honey
preparation: None

wasabi paste
quantity: 
measurement: None
descriptor: None
ingredient without descriptor: wasabi paste
preparation: None

1 tablespoon rice wine

2 tablespoons ketchup
quantity: 2
measurement: tablespoons
descriptor: None
ingredient without descriptor: ketchup
preparation: None

1/4 teaspoon ground ginger
quantity: 1/4
measurement: teaspoon
descriptor: ground
ingredient without descriptor: ginger
preparation: None

12 ounces cooked shrimp, tails removed
quantity: 12
measurement: ounces
descriptor: cooked
ingredient without descriptor: shrimp
preparation: tails removed

1 tablespoon vegetable oil
quantity: 1
measurement: tablespoon
descriptor: vegetable
ingredient without descriptor: oil
preparation: None

4 cloves garlic, minced
quantity: 4
measurement: cloves
descriptor: None
ingredient without descriptor: garlic
preparation: minced

4 tablespoons water
quantity: 4
measurement: tablespoons
descriptor: None
ingredient without descriptor: water
preparation: None

2 teaspoons cornstarch
quantity: 2
measurement: teaspoons
descriptor: None
ingredient without descriptor: cornstarch
preparation: None

1/4 cup sliced green onions
quant

1/2 (4.5 ounce) can sliced mushrooms
quantity: 1/2 (4.5 ounce)
measurement: can
descriptor: sliced
ingredient without descriptor: mushrooms
preparation: None

1 cup crushed potato chips
quantity: 1
measurement: cup
descriptor: crushed
ingredient without descriptor: potato chips
preparation: None

1 (12 ounce) package egg noodles
quantity: 1 (12 ounce)
measurement: package
descriptor: None
ingredient without descriptor: egg noodles
preparation: None

2 (10.75 ounce) cans condensed cream of mushroom soup
quantity: 2 (10.75 ounce)
measurement: cans
descriptor: condensed
ingredient without descriptor: cream of mushroom soup
preparation: None

1/4 cup chopped onion
quantity: 1/4
measurement: cup
descriptor: chopped
ingredient without descriptor: onion
preparation: None

1 cup frozen green peas
quantity: 1
measurement: cup
descriptor: frozen green
ingredient without descriptor: peas
preparation: None

2 (5 ounce) cans tuna, drained
quantity: 2 (5 ounce)
measurement: cans
descriptor: None
ing

1/2 cup chicken broth
quantity: 1/2
measurement: cup
descriptor: None
ingredient without descriptor: chicken broth
preparation: None

1 pound fresh shrimp, peeled and deveined
quantity: 1
measurement: pound
descriptor: fresh
ingredient without descriptor: shrimp
preparation: peeled and deveined

1 tablespoon chopped fresh basil
quantity: 1
measurement: tablespoon
descriptor: chopped fresh
ingredient without descriptor: basil
preparation: None

1 lemon, juiced
quantity: 1
measurement: None
descriptor: None
ingredient without descriptor: lemon
preparation: juiced

1/4 cup white wine
quantity: 1/4
measurement: cup
descriptor: white
ingredient without descriptor: wine
preparation: None

2 teaspoons freshly ground black pepper
quantity: 2
measurement: teaspoons
descriptor: freshly ground black
ingredient without descriptor: pepper
preparation: None

1/2 teaspoon lemon zest
quantity: 1/2
measurement: teaspoon
descriptor: None
ingredient without descriptor: lemon zest
preparation: None

salt 

In [23]:
sorted(seafood_ingredients_freq_dict.items(), key=lambda pair: pair[1], reverse=True);

In [24]:
seafood_ingredients_preparation_dict;

In [25]:
ingredients_categorized = {}
ingredients_categorized['vegetarian_protein'] = {}
ingredients_categorized['vegetarian_protein']['ingredients'] = vegetarian_protein_ingredients_freq_dict
ingredients_categorized['vegetarian_protein']['measurement'] = vegetarian_protein_ingredients_measurement_dict
ingredients_categorized['vegetarian_protein']['descriptor'] = vegetarian_protein_ingredients_descriptor_dict
ingredients_categorized['vegetarian_protein']['preparation'] = vegetarian_protein_ingredients_preparation_dict
ingredients_categorized['meat'] = {}
ingredients_categorized['meat']['ingredients'] = meat_ingredients_freq_dict
ingredients_categorized['meat']['measurement'] = meat_ingredients_measurement_dict
ingredients_categorized['meat']['descriptor'] = meat_ingredients_descriptor_dict
ingredients_categorized['meat']['preparation'] = meat_ingredients_preparation_dict
ingredients_categorized['seafood'] = {}
ingredients_categorized['seafood']['ingredients'] = seafood_ingredients_freq_dict
ingredients_categorized['seafood']['measurement'] = seafood_ingredients_measurement_dict
ingredients_categorized['seafood']['descriptor'] = seafood_ingredients_descriptor_dict
ingredients_categorized['seafood']['preparation'] = seafood_ingredients_preparation_dict

In [26]:
# save scraped data
with open('ingredients_categorized.pickle', 'wb') as file:
    pickle.dump(ingredients_categorized, file, protocol=pickle.HIGHEST_PROTOCOL)
file.closed

True

In [27]:
# check
with open('ingredients_categorized.pickle', 'rb') as file:
    ingredients_categorized = pickle.load(file)

sorted(ingredients_categorized['meat']['ingredients'].items(), key=lambda pair: pair[1], reverse=True)

[('onion', 71),
 ('pepper', 70),
 ('garlic', 65),
 ('salt', 62),
 ('sugar', 35),
 ('olive oil', 32),
 ('oregano', 27),
 ('water', 27),
 ('butter', 25),
 ('salt and pepper', 25),
 ('cumin', 23),
 ('Worcestershire sauce', 23),
 ('garlic powder', 22),
 ('beef', 21),
 ('parsley', 21),
 ('flour', 20),
 ('skinless', 20),
 ('thyme', 19),
 ('cayenne pepper', 18),
 ('tomatoes', 18),
 ('oil', 18),
 ('chicken broth', 17),
 ('celery', 16),
 ('carrots', 16),
 ('ketchup', 15),
 ('milk', 15),
 ('chili powder', 15),
 ('soy sauce', 14),
 ('paprika', 14),
 ('bell pepper', 13),
 ('beef broth', 13),
 ('wine', 13),
 ('bread crumbs', 12),
 ('onions', 12),
 ('tomato sauce', 12),
 ('egg', 12),
 ('bacon', 12),
 ('lamb', 11),
 ('eggs', 10),
 ('basil', 10),
 ('potatoes', 10),
 ('Parmesan cheese', 10),
 ('cream', 10),
 ('Cheddar cheese', 10),
 ('ginger', 9),
 ('vinegar', 8),
 ('leaf', 8),
 ('cilantro', 8),
 ('tomato paste', 8),
 ('garlic salt', 8),
 ('cinnamon', 8),
 ('chops', 8),
 ('leaves', 7),
 ('pepper flakes